# MQTT Publish 測試應用程式

這個 notebook 示範如何使用 paho-mqtt 套件來發佈 MQTT 訊息。

In [ ]:
# 導入必要的套件
import paho.mqtt.client as mqtt
import time
import json

In [ ]:
# MQTT Broker 設定（連接到本地 Raspberry Pi 的 MQTT Broker）
BROKER = "localhost"  # 或使用 "127.0.0.1"
PORT = 1883  # 標準 MQTT 埠，如果您的 Broker 使用不同埠號請修改
TOPIC = "客廳/溫度"  # 您可以修改這個主題名稱

# 建立 MQTT 客戶端（使用新的回調 API 版本 2）
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# 連線回調函數（VERSION2 API）
def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code.is_failure:
        print(f"❌ 連線失敗，錯誤代碼: {reason_code}")
    else:
        print(f"✅ 成功連接到 MQTT Broker: {BROKER}")

# 設定連線回調
client.on_connect = on_connect

# 連接到 Broker
print(f"正在連接到 {BROKER}...")
client.connect(BROKER, PORT, 60)
client.loop_start()  # 開始背景執行緒處理訊息

# 等待連線建立
time.sleep(1)

In [ ]:
# 發佈簡單的文字訊息
message = "Hello MQTT from Raspberry Pi!"
result = client.publish(TOPIC, message, qos=1)

if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print(f"✅ 訊息已發佈到主題: {TOPIC}")
    print(f"   訊息內容: {message}")
else:
    print(f"❌ 發佈失敗，錯誤代碼: {result.rc}")

In [ ]:
# 發佈 JSON 格式的訊息
data = {
    "device": "Raspberry Pi",
    "temperature": 25.5,
    "humidity": 60,
    "timestamp": time.time()
}

json_message = json.dumps(data)
result = client.publish(TOPIC, json_message, qos=1)

if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print(f"✅ JSON 訊息已發佈到主題: {TOPIC}")
    print(f"   訊息內容: {json_message}")
else:
    print(f"❌ 發佈失敗，錯誤代碼: {result.rc}")

In [ ]:
# 連續發佈多筆訊息（測試用）
print("開始連續發佈訊息...")
for i in range(5):
    message = f"測試訊息 #{i+1} - 時間: {time.strftime('%Y-%m-%d %H:%M:%S')}"
    result = client.publish(TOPIC, message, qos=1)
    
    if result.rc == mqtt.MQTT_ERR_SUCCESS:
        print(f"✅ [{i+1}/5] {message}")
    else:
        print(f"❌ [{i+1}/5] 發佈失敗")
    
    time.sleep(1)  # 等待 1 秒

print("\n✅ 所有訊息發佈完成！")

In [ ]:
# 關閉連線
client.loop_stop()
client.disconnect()
print("✅ MQTT 連線已關閉")